# CS470 Introduction to Artificial Intelligence
## Deep Learning Practice 
#### TA. Yechan Hwang
---

### Agenda for this practice
#### 1. Shakespeare dataset
#### 2. GRU Model
#### 3. Generating texts
---
<br/>
<br/>
<br/>

## 6-1. Text generation with an RNN 
In this practice, we will learn how to generate text using a character-based RNN. We will practice with a dataset of **Shakespeare's writing** (from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)). We will train a model when given a sequence of characters from this data, that predicts the next character in the sequence. For example, when given the characters 'togethe', trained model will predict 'r' for the next character. Longer sequences of text can be generated by calling the model repeatedly.

The following is sample output when the model in this practice trained for 30 epochs, and started with the character 'Q'.


<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>


While most of the sentences are grammatically correct, they do not make sense. But the model seems to have learned some attributes.

- Before the training, the model can't know the style of the training data. 
- But after training, the structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

#### Import libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

#### Download the Shakespeare dataset
Run the following lines to download data for training.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
print(path_to_file)

1122304/1115394 [==============================] - 0s 0us/step
/root/.keras/datasets/shakespeare.txt


#### Read the data
First, let's take a look at the length of the data.

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print ('Length of text: {} characters'.format(len(text)))
#from shakespeare.txt

Length of text: 1115394 characters


Here *length of text* is the number of characters in it. We have more than ten million characters.<br/>
Also, we can check the first 250 characters in training text.

In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



Our dataset has the format of the screenplay.

And how many unique characters are there? Let's check it.

In [5]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [6]:
vocab

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

There are some special symbols and characters (including lowercase and uppercase letters)

#### Vectorize the text
Before training, we need to **map all the characters in the dataset to a numerical representation**. 

We will create two lookup tables: 
- one for mapping **characters to numbers**
- another for **numbers to characters**

In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# 1D integer vector for all characters in the text data
text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print('{')
for char,_ in zip(char2idx, range(65)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  'H' :  20,
  'I' :  21,
  'J' :  22,
  'K' :  23,
  'L' :  24,
  'M' :  25,
  'N' :  26,
  'O' :  27,
  'P' :  28,
  'Q' :  29,
  'R' :  30,
  'S' :  31,
  'T' :  32,
  'U' :  33,
  'V' :  34,
  'W' :  35,
  'X' :  36,
  'Y' :  37,
  'Z' :  38,
  'a' :  39,
  'b' :  40,
  'c' :  41,
  'd' :  42,
  'e' :  43,
  'f' :  44,
  'g' :  45,
  'h' :  46,
  'i' :  47,
  'j' :  48,
  'k' :  49,
  'l' :  50,
  'm' :  51,
  'n' :  52,
  'o' :  53,
  'p' :  54,
  'q' :  55,
  'r' :  56,
  's' :  57,
  't' :  58,
  'u' :  59,
  'v' :  60,
  'w' :  61,
  'x' :  62,
  'y' :  63,
  'z' :  64,

}


In [9]:
print(len(text_as_int))
print(text_as_int)

1115394
[18 47 56 ... 45  8  0]


<br/><br/>
Also let's check how the first 13 characters from the dataset text are mapped to integers.

In [10]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


#### The prediction task
Our goal is when given a character or a sequence of characters, to predict **the most probable following character.** Therefore, the **input to the model will be a sequence of characters** and the model will learn to **predict the output : the following character at each time step**.



#### Create training examples and targets
For now, divide the text into training sequences. Each input sequence will contain `seq_length` characters from the text. For each input sequence, the corresponding targets contain the same length of text, but shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, let's say that `seq_length` is 4 and our training text is "HELLO".

In this example, **the input sequence would be "HELL", and the target sequence "ELLO"**.

<img src="https://github.com/mikodham/CS470/blob/main/Lab3/May%2013/images/teacher_forcing.png?raw=1" alt="Drawing" style="width: 700px;"/>

To do this, first use the [`tf.data.Dataset.from_tensor_slices`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#from_tensor_slices) function to convert the text vector into a stream of character indices.
- `tf.data.Dataset.from_tensor_slices`: Creates a Dataset whose elements are slices of the given tensors.


In [11]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Make char dataset (in the form of integer)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(str(i.numpy())+" : "+str(idx2char[i.numpy()]))
    

18 : F
47 : i
56 : r
57 : s
58 : t


In [12]:
# Make sequences with sequence length +1
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

print(str(len(sequences))+" sequences of length "+str(seq_length+1))
print()

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))
    print(len(idx2char[item.numpy()]))

11043 sequences of length 101

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
101
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
101
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
101
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
101
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'
101


<br/><br/>

Now we have to convert above text into input data and target data. Note that target data must be shifted one character to the right.

To do this, we will use `tf.data.Dataset.map`. When we give some function to `tf.data.Dataset.map` as a parameter, it will apply the function to all elements and then return them.

In [13]:
def plus_1(x):
    return x+1
    
temp_dataset = tf.data.Dataset.range(1, 6)  # ==> [ 1, 2, 3, 4, 5 ]
print(list(temp_dataset.as_numpy_iterator()))
temp_dataset = temp_dataset.map(plus_1)
print(list(temp_dataset.as_numpy_iterator()))

[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]


In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
print(len(dataset))

11043


In [15]:
for input_example, target_example in dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


<br/>

During the training, **each index of these vectors are processed as one time step**. For the input at time step 0, the model receives the index for "F" and tries to predict the index for "i" as the next character. At the next timestep, it does the same thing but the **RNN considers the previous step context in addition to the current input character**.

In [16]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


#### Create training batches
We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to **shuffle the data and pack it into batches**.

[`tf.data.Dataset.shuffle`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#shuffle)(buffer_size, seed=None, reshuffle_each_iteration=None) : Randomly shuffles the elements of this dataset.

[`tf.data.Dataset.batch`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#batch)(batch_size, drop_remainder=False) : Combines consecutive elements of this dataset into batches.

Note that `tf.data.Dataset.shuffle` **doesn't shuffle characters in each sentence**, but the sentences in dataset will be shuffled by sentences.

<img src="https://github.com/mikodham/CS470/blob/main/Lab3/May%2013/images/shuffle1.png?raw=1" alt="Drawing" style="width: 600px;"/>

<br/>
<br/>

<img src="https://github.com/mikodham/CS470/blob/main/Lab3/May%2013/images/shuffle2.png?raw=1" alt="Drawing" style="width: 600px;"/>

In [17]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

# Shuffle the data and create batches (1 data = (100, 100) ==> 0:99, 1:100)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset
#each batch has 64 sentences, each sentence has 100 characters

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

<br/><br/>

We can see that each batch has 64 input sentences (each has 100 characters) and 64 target sentences (each has 100 characters). 

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch, target_example_batch)

tf.Tensor(
[[46 43 56 ... 45  1 47]
 [41 53 59 ... 32 46 43]
 [ 1 40 43 ... 57  1 58]
 ...
 [44 53 56 ... 59 56  0]
 [46 43 43 ... 43  1 61]
 [53 59  1 ... 53 51 54]], shape=(64, 100), dtype=int64) tf.Tensor(
[[43 56  1 ...  1 47 52]
 [53 59 50 ... 46 43  1]
 [40 43 47 ...  1 58 46]
 ...
 [53 56 58 ... 56  0 42]
 [43 43  1 ...  1 61 43]
 [59  1 42 ... 51 54 43]], shape=(64, 100), dtype=int64)


<br/><br/>

Also we can see that all the target sentences are shifted one character to the right .

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch[0], target_example_batch[0])

tf.Tensor(
[ 0 21  6  1 57 47 56  2  1 52 43  5 43 56  1 39  1 61 46 47 58  8  0  0
 24 33 15 17 26 32 21 27 10  0 13 52 42  1 52 53 58  1 39  1 48 53 58  1
 53 44  1 32 56 39 52 47 53  1 47 52  1 63 53 59 56  1 51 53 59 58 46 10
  0 32 56 39 52 47 53  1 47 57  1 41 46 39 52 45 43 42  1 47 52 58 53  1
 24 59 41 43], shape=(100,), dtype=int64) tf.Tensor(
[21  6  1 57 47 56  2  1 52 43  5 43 56  1 39  1 61 46 47 58  8  0  0 24
 33 15 17 26 32 21 27 10  0 13 52 42  1 52 53 58  1 39  1 48 53 58  1 53
 44  1 32 56 39 52 47 53  1 47 52  1 63 53 59 56  1 51 53 59 58 46 10  0
 32 56 39 52 47 53  1 47 57  1 41 46 39 52 45 43 42  1 47 52 58 53  1 24
 59 41 43 52], shape=(100,), dtype=int64)


#### Build The GRU Model
We will use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

- `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;
- [`tf.keras.layers.GRU`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/GRU): A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)
<img src=https://miro.medium.com/max/2400/1*dhq14CzJijlqjf7IlDB0uw.png>

<img src=https://lh3.googleusercontent.com/proxy/sij4sv35kGVFJvWw-lwYr71iFEhXzGSxodnLNyxwKd7dpZvggXDkZoVojzIiHFgvc73qYMHAZhw0gKW-Bp-b7IVnh25hTbfpOo0lNShS9q0-3x3ujUTO8uJ1adJ1bg2p>

 
- `tf.keras.layers.Dense`: The output layer, with vocab_size outputs.

##### About the GRU

GRU is a variation of LSTM. GRU has some different attributes compared to vanilla LSTM.

- The two state vectors $c_t$ and $h_t$ in the LSTM Cell are merged into one vector $h_t$.
- There is only one gate controller $z_t$ that controls all input gates.
- There is no output gate and the state vector $h_t$ is the output of GRU.

You can see details about the GRU at the this [link](https://arxiv.org/abs/1406.1078).
In this practice, we will use GRU since its operation is faster than LSTM and it has fewer parameters.

In [20]:
# Length of the vocabulary in chars
vocab_size = len(vocab) #65

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [21]:
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Embedding, GRU, Dense 

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    ###################
    #                 #
    # write your code #
    #                 # 
    model=Sequential([
      Embedding(
          vocab_size,embedding_dim,
          batch_input_shape = [batch_size,None]
      ),
      GRU(rnn_units, 
          return_sequences=True,
          stateful=True,
          recurrent_initializer='glorot_uniform'
      ),
      Dense(vocab_size)
    ])
    return model
    ###################

In [22]:
model = build_model(
    vocab_size=len(vocab), 
    embedding_dim = embedding_dim, 
    rnn_units = rnn_units, 
    batch_size=BATCH_SIZE
)
    ###################
    #                 #
    # write your code #
    #                 # 
    ###################

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

<img src=https://www.tensorflow.org/text/tutorials/images/text_generation_training.png>

#### Try the untrained model
Now we will run the untrained model to see how it behaves.

First let's check the shape of the output.

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") 

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


<br/><br/>

Also we can check the model's prediction by probability distribution. For example, we can see the model's prediction for the first sentence's fifth character.

(Note that since the current model is not trained yet.)

In [24]:
example_batch_predictions[0][5]

<tf.Tensor: shape=(65,), dtype=float32, numpy=
array([-1.27089042e-02,  1.44095663e-02,  2.00382918e-02,  1.11477487e-02,
       -1.64984353e-03, -9.28282458e-03, -1.01572452e-02,  7.01599894e-03,
        1.92178041e-02,  9.12852213e-03,  3.12883081e-03,  2.84779491e-03,
        5.52502507e-03,  7.72807281e-04, -1.69514120e-03, -5.99782215e-03,
        3.59804952e-03, -1.77651481e-03,  7.42385443e-03,  6.32357830e-03,
       -7.25982385e-03, -1.05561735e-02,  1.38615975e-02, -6.61300682e-03,
        7.52078649e-03, -1.37021672e-03, -2.67638499e-03, -1.21365883e-03,
       -9.33160726e-03, -1.13348076e-02, -5.06472494e-03,  2.01825751e-03,
        3.12439655e-03,  5.47962729e-03,  5.94117399e-03,  1.66756730e-03,
        1.42683797e-02, -1.50501141e-02,  1.22760679e-03, -7.77127594e-03,
       -1.46131814e-02, -1.56429992e-03, -7.85714202e-03,  4.62916167e-03,
        8.03106185e-03, -4.87098796e-03,  2.68872059e-03, -1.27286045e-02,
       -7.39303092e-03,  2.31492799e-04,  1.17489723e

And in our model, the sequence length of the input is 100 but the model can be run on inputs of any length.

This is an advantage of the recurrent neural network which can handle inputs of variable length.

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model, we need to sample from the output distribution to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Try it for the first example in the batch:

In [26]:
# num_samples : determines how many characters to sample at each iteration

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
print(sampled_indices)

tf.Tensor(
[[35]
 [22]
 [ 4]
 [20]
 [38]
 [ 0]
 [56]
 [ 7]
 [49]
 [55]
 [ 2]
 [25]
 [57]
 [56]
 [54]
 [22]
 [19]
 [52]
 [25]
 [55]
 [35]
 [34]
 [ 4]
 [38]
 [34]
 [47]
 [50]
 [ 9]
 [27]
 [28]
 [ 6]
 [42]
 [ 6]
 [19]
 [ 0]
 [ 6]
 [31]
 [34]
 [ 8]
 [45]
 [49]
 [56]
 [12]
 [44]
 [ 5]
 [55]
 [51]
 [ 3]
 [21]
 [16]
 [24]
 [11]
 [54]
 [41]
 [ 1]
 [25]
 [ 9]
 [15]
 [40]
 [45]
 [44]
 [ 9]
 [43]
 [63]
 [27]
 [57]
 [17]
 [ 0]
 [ 5]
 [12]
 [ 5]
 [30]
 [13]
 [38]
 [33]
 [ 2]
 [55]
 [ 5]
 [51]
 [32]
 [57]
 [34]
 [13]
 [ 7]
 [61]
 [46]
 [47]
 [40]
 [12]
 [26]
 [15]
 [41]
 [55]
 [38]
 [62]
 [12]
 [22]
 [43]
 [48]
 [38]], shape=(100, 1), dtype=int64)


This gives us a prediction for the next character index at each timestep.

Now to see the predicted sentence of our untrained model, we will squeeze the `sampled_indices` and convert them into characters.
- [`tf.squeeze`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/squeeze): Removes dimensions of size 1 from the shape of a tensor.

In [27]:
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

print(sampled_indices.shape)
print(sampled_indices)

(100,)
[35 22  4 20 38  0 56  7 49 55  2 25 57 56 54 22 19 52 25 55 35 34  4 38
 34 47 50  9 27 28  6 42  6 19  0  6 31 34  8 45 49 56 12 44  5 55 51  3
 21 16 24 11 54 41  1 25  9 15 40 45 44  9 43 63 27 57 17  0  5 12  5 30
 13 38 33  2 55  5 51 32 57 34 13  7 61 46 47 40 12 26 15 41 55 38 62 12
 22 43 48 38]


After sqeezing the `sampled_indices`, we got 1D vector that contains indicies of predicted characters.

Let's decode this vector to see the text predicted by this untrained model.

In [28]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'brother did love Juliet,\nAnd you tell me that he shall die for it.\n\nANGELO:\nHe shall not, Isabel, if'

Next Char Predictions: 
 "WJ&HZ\nr-kq!MsrpJGnMqWV&ZVil3OP,d,G\n,SV.gkr?f'qm$IDL;pc M3Cbgf3eyOsE\n'?'RAZU!q'mTsVA-whib?NCcqZx?JejZ"


<br/>

Since our model is not trained yet, it seems to just predict next character randomly.

#### Train the model
At this point the problem can be treated as a standard classification problem. **Given the previous RNN state and the input character at each time step, our model must predict the next character.**

#### Compile the model
We will use `tf.keras.losses.sparse_categorical_crossentropy` loss function since it works well for classification problem.

Since our model returns logits, we need to set the `from_logits` flag.

In [29]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Get loss value from untrained model
example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1736755


In [30]:
model.compile(optimizer='adam', 
              loss=loss)

#### Configure checkpoints
Use a [`tf.keras.callbacks.ModelCheckpoint`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks/ModelCheckpoint) to ensure that checkpoints are saved during training:

In [31]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [32]:
history = model.fit(dataset, 
                    epochs=15, 
                    callbacks=[checkpoint_callback])

Epoch 1/15
172/172 [==============================] - 12s 56ms/step - loss: 2.6871
Epoch 2/15
172/172 [==============================] - 10s 56ms/step - loss: 1.9543
Epoch 3/15
172/172 [==============================] - 11s 57ms/step - loss: 1.6899
Epoch 4/15
172/172 [==============================] - 11s 59ms/step - loss: 1.5428
Epoch 5/15
172/172 [==============================] - 11s 60ms/step - loss: 1.4546
Epoch 6/15
172/172 [==============================] - 11s 58ms/step - loss: 1.3949
Epoch 7/15
172/172 [==============================] - 11s 58ms/step - loss: 1.3491
Epoch 8/15
172/172 [==============================] - 11s 58ms/step - loss: 1.3111
Epoch 9/15
172/172 [==============================] - 11s 57ms/step - loss: 1.2750
Epoch 10/15
172/172 [==============================] - 11s 58ms/step - loss: 1.2418
Epoch 11/15
172/172 [==============================] - 11s 59ms/step - loss: 1.2101
Epoch 12/15
172/172 [==============================] - 11s 57ms/step - loss: 1.1775
E

#### Generate text
We will restore the latest checkpoint. Then, to keep this prediction step simple, we will use 1 for batch size.

(To run the model with a different `batch_size`, we need to rebuild the model with different batch size and restore the weights from the checkpoint.)

By the codes below, we can check the path that contains weights for the lastest model and load it.

In [39]:
# Rebuild the model by changing batch size (=1) to predict new text
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# Load the weight of the model we trained 
# model.load_weights('./saved_ckpt/ckpt_15')

# Change the batch size from 64 to 1
model.build(tf.TensorShape([1, None]))

NotFoundError: ignored

In [ ]:
model.summary()

#### The prediction loop

The following code block generates the text:

1. Start by choosing a **start string** and initialize the RNN hidden state for the first iteration.
2. Set the number of characters to generate.
3. Get the **prediction distribution of the next character using the start string and hidden state**.
4. Smaple an index of the predicted character using a multinomial distribution of the first iteration. 
5. Use this predicted character as our next input to the model.
6. Repeat step 3-5 until we get the number of characters we set.

**Note that the RNN hidden state returned by the model is fed back into the model and hidden state will become more complex as the prediction loop repeats.**
In other words, after predicting the a word, the modified RNN states are again fed back into the model, which is how the model learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](https://www.tensorflow.org/tutorials/text/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize and make paragraphs, and it imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [36]:
def generate_text(model, start_string, num_generate,temperature):
    # Evaluation step (generating text using the learned model)

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [37]:
# Low temperatures results in more predictable text.
# Higher temperatures results in more surprising text.
# Experiment to find the best setting.

print(generate_text(model, start_string="ROMEO:", num_generate = 1000, temperature=1.0))

AttributeError: ignored

<br/><br/>
The easiest thing you can do to improve the results is to train it for longer (e.g. try EPOCHS=30).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.